In [9]:
import os
import re
import pandas as pd

# Ruta de la carpeta raíz
carpeta_raiz = r"C:\Users\AUX ADMINISTRATIVO\Documents\TOBAR\BOOTCAMP PROGRAMACION\Phyton proyecto\07 JULIO"

# Obtener todas las subcarpetas como carpetas esperadas
carpetas_esperadas = [nombre for nombre in os.listdir(carpeta_raiz)
                      if os.path.isdir(os.path.join(carpeta_raiz, nombre))]

print("📁 Carpetas esperadas detectadas:")
for carpeta in carpetas_esperadas:
    print(" -", carpeta)

# Diccionario para almacenar archivos por paciente
pacientes_archivos = {}

# Recorrer los archivos PDF en cada subcarpeta
for subcarpeta in carpetas_esperadas:
    ruta_subcarpeta = os.path.join(carpeta_raiz, subcarpeta)
    
    for archivo in os.listdir(ruta_subcarpeta):
        if archivo.endswith(".pdf"):
            ruta_archivo = os.path.join(ruta_subcarpeta, archivo)
            
            # Extraer datos del nombre del archivo con regex
            match = re.match(r"(.+?)_(.+?)_(\d+)_.*\.pdf", archivo)
            if match:
                aseguradora = match.group(1).strip()
                nombre = match.group(2).strip()
                documento = match.group(3).strip()

                clave = (documento, aseguradora, nombre)

                if clave not in pacientes_archivos:
                    pacientes_archivos[clave] = {}

                pacientes_archivos[clave][subcarpeta] = archivo

# Crear el DataFrame con columnas dinámicas
df = pd.DataFrame([
    {
        "DOCUMENTO": doc,
        "ASEGURADORA": aseg,
        "NOMBRE COMPLETO": nombre,
        **{carpeta: archivos.get(carpeta, "") for carpeta in carpetas_esperadas}
    }
    for (doc, aseg, nombre), archivos in pacientes_archivos.items()
])

# Mostrar tabla general
print("\n📄 Tabla de archivos encontrados:")
print(df.to_string(index=False))

# Identificar pacientes con archivos faltantes
df["FALTANTES"] = df.apply(
    lambda row: [carpeta for carpeta in carpetas_esperadas if row[carpeta] == ""], axis=1
)
faltantes_df = df[df["FALTANTES"].apply(lambda x: len(x) > 0)]

# Mostrar tabla de faltantes
print("\n❌ Pacientes con archivos faltantes:")
if not faltantes_df.empty:
    print(faltantes_df[["DOCUMENTO", "ASEGURADORA", "NOMBRE COMPLETO", "FALTANTES"]].to_string(index=False))
else:
    print("✅ Todos los pacientes tienen archivos en todas las carpetas.")


📁 Carpetas esperadas detectadas:
 - 1 COMP
 - 2 HUELLA
 - 3 INFORME
 - 4.1 PSICO
 - 4.2 NUTR
 - 4.3 T SOC
 - 5 LABS
 - 6 NOTAS
 - 7 FARM
 - 8 CEDULA

📄 Tabla de archivos encontrados:
 DOCUMENTO ASEGURADORA                  NOMBRE COMPLETO                                                            1 COMP                                                            2 HUELLA                                                            3 INFORME                                                            4.1 PSICO                                                            4.2 NUTR                                                            4.3 T SOC                                                            5 LABS                                                            6 NOTAS                                                            7 FARM                                                            8 CEDULA
   4640199 ASMET SALUD          ARANDA MOSQUERA ABSALON            ASMET SALUD_ARANDA